In [1]:
pip install opencv-python numpy pandas scikit-image scipy openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 15.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [openpyxl]5/6 [openpyxl]age]
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install rembg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [rembg]
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install rembg onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [5]:
# -----------------------------------------------------------------------------
# Image Feature Extractor v3.1 (Bugfix for Grayscale Conversion)
# -----------------------------------------------------------------------------
# This version corrects a bug where the original image was being converted
# to grayscale instead of the background-removed image.
#
# Required installation: pip install rembg onnxruntime Pillow opencv-python ...
# -----------------------------------------------------------------------------

import os
import cv2
import numpy as np
import pandas as pd
from skimage.measure import label, regionprops
from skimage.feature import graycomatrix, graycoprops
from scipy.stats import skew, kurtosis
from skimage.measure import shannon_entropy
import rembg
from PIL import Image

# --- Configuration: Define file paths ---
INPUT_DIR = '/home/sajad/Sajad_test/Banana/Red/d1'
BG_REMOVED_DIR = '/home/sajad/Sajad_test/Banana/Red/d1/output_bg_removed'
GRAYSCALE_DIR = '/home/sajad/Sajad_test/Banana/Red/d1/output_grayscale'
EXCEL_PATH = '/home/sajad/Sajad_test/Banana/Red/d1/image_features_v3_1.xlsx'

def create_directories():
    """Create output directories if they don't exist."""
    os.makedirs(BG_REMOVED_DIR, exist_ok=True)
    os.makedirs(GRAYSCALE_DIR, exist_ok=True)

def extract_fourier_features(roi, num_rings=10):
    # This function remains unchanged.
    if roi.size == 0:
        fourier_features = {f'fourier_ring_{i+1}': 0 for i in range(num_rings)}
        fourier_features['fourier_energy_concentration_25'] = 0
        return fourier_features
    f_transform = np.fft.fft2(roi)
    f_transform_shifted = np.fft.fftshift(f_transform)
    magnitude_spectrum = np.abs(f_transform_shifted)
    h, w = magnitude_spectrum.shape
    center_h, center_w = h // 2, w // 2
    y, x = np.ogrid[:h, :w]
    distance_map = np.sqrt((x - center_w)**2 + (y - center_h)**2)
    max_radius = np.sqrt(center_h**2 + center_w**2)
    radius_step = max_radius / num_rings
    fourier_features = {}
    total_energy = np.sum(magnitude_spectrum**2)
    for i in range(num_rings):
        inner_radius = i * radius_step
        outer_radius = (i + 1) * radius_step
        ring_mask = (distance_map >= inner_radius) & (distance_map < outer_radius)
        ring_sum = np.sum(magnitude_spectrum[ring_mask])
        fourier_features[f'fourier_ring_{i+1}'] = ring_sum
    radius_25_percent = max_radius * 0.25
    energy_mask_25 = distance_map < radius_25_percent
    energy_in_25 = np.sum(magnitude_spectrum[energy_mask_25]**2)
    fourier_features['fourier_energy_concentration_25'] = energy_in_25 / total_energy if total_energy > 0 else 0
    return fourier_features

def extract_features(image_path):
    """
    Processes a single image to remove background, convert to grayscale,
    and extract a full suite of features.
    """
    filename = os.path.basename(image_path)
    
    # --- 1. Background Removal using Deep Learning (rembg) ---
    try:
        input_image_pil = Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Warning: Could not open image {filename} with PIL. Skipping. Error: {e}")
        return None

    output_image_pil = rembg.remove(input_image_pil)
    
    # Convert PIL (RGBA) to OpenCV format (BGR)
    bg_removed_image_rgba = np.array(output_image_pil)
    bg_removed_image_rgb = bg_removed_image_rgba[:, :, :3]
    bg_removed_image_bgr = cv2.cvtColor(bg_removed_image_rgb, cv2.COLOR_RGB2BGR)

    # Create the final mask from the alpha channel
    alpha_channel = bg_removed_image_rgba[:, :, 3]
    final_mask = (alpha_channel > 0).astype(np.uint8) * 255

    # Apply mask to make background black (for visualization and saving)
    bg_removed_image_bgr[final_mask == 0] = [0, 0, 0]
    
    # Save the background-removed image
    cv2.imwrite(os.path.join(BG_REMOVED_DIR, filename), bg_removed_image_bgr)

    # --- 2. Grayscale Conversion (CORRECTED) ---
    # Convert the BACKGROUND-REMOVED image to grayscale, not the original.
    grayscale_image = cv2.cvtColor(bg_removed_image_bgr, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(os.path.join(GRAYSCALE_DIR, filename), grayscale_image)

    # --- 3. Feature Extraction ---
    # All subsequent operations correctly use the 'grayscale_image' and 'final_mask'
    features = {'filename': filename}
    object_pixels_gray = grayscale_image[final_mask == 255]
    
    labeled_mask = label(final_mask)
    props = regionprops(labeled_mask, intensity_image=grayscale_image)
    
    if props:
        prop = props[0]
        y0, x0, y1, x1 = prop.bbox
        
        # Shape Features
        features['area'] = prop.area
        features['perimeter'] = prop.perimeter
        features['eccentricity'] = prop.eccentricity
        features['solidity'] = prop.solidity
        features['equivalent_diameter'] = prop.equivalent_diameter
        
        # Statistical Features
        if object_pixels_gray.size > 0:
            features['mean_intensity'] = np.mean(object_pixels_gray)
            features['std_dev_intensity'] = np.std(object_pixels_gray)
            features['skewness'] = skew(object_pixels_gray)
            features['kurtosis'] = kurtosis(object_pixels_gray)
        else:
            features.update({k: 0 for k in ['mean_intensity', 'std_dev_intensity', 'skewness', 'kurtosis']})
        
        # Texture Features (GLCM)
        roi = grayscale_image[y0:y1, x0:x1]
        if roi.size > 0:
            glcm = graycomatrix(roi, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
            features['contrast'] = graycoprops(glcm, 'contrast')[0, 0]
            features['correlation'] = graycoprops(glcm, 'correlation')[0, 0]
            features['energy'] = graycoprops(glcm, 'energy')[0, 0]
            features['homogeneity'] = graycoprops(glcm, 'homogeneity')[0, 0]
            features['entropy'] = shannon_entropy(roi)
        else:
            features.update({k: 0 for k in ['contrast', 'correlation', 'energy', 'homogeneity', 'entropy']})

        # Fourier Features
        fourier_features = extract_fourier_features(roi)
        features.update(fourier_features)
    else:
        print(f"Warning: No object found in {filename} after background removal.")
        return None
            
    return features

def main():
    """Main function to run the image processing pipeline."""
    create_directories()
    all_features = []
    
    if not os.path.isdir(INPUT_DIR) or not os.listdir(INPUT_DIR):
        print(f"Error: Input directory '{INPUT_DIR}' not found or is empty.")
        return

    print("Starting image processing with Deep Learning background removal (v3.1)...")
    
    image_files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    
    for i, filename in enumerate(image_files):
        print(f"Processing {i+1}/{len(image_files)}: {filename}...")
        try:
            image_path = os.path.join(INPUT_DIR, filename)
            features = extract_features(image_path)
            if features:
                all_features.append(features)
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")

    if not all_features:
        print("No features were extracted. Exiting.")
        return

    df = pd.DataFrame(all_features)
    df.to_excel(EXCEL_PATH, index=False, engine='openpyxl')
    
    print(f"\nProcessing complete.")
    print(f"Features saved to: {EXCEL_PATH}")
    print(f"Background-removed images saved in: {BG_REMOVED_DIR}")
    print(f"Grayscale images (from bg-removed) saved in: {GRAYSCALE_DIR}")

if __name__ == "__main__":
    main()

Starting image processing with Deep Learning background removal (v3.1)...
Processing 1/11: R- pvc- r2-d1.jpg...
Processing 2/11: R- p- r2-d1.jpg...
Processing 3/11: R- p- r1-d1.jpg...
Processing 4/11: R- pe- r2-d1.jpg...
Processing 5/11: R- pvc- r1-d1.jpg...
Processing 6/11: R- N- r3-d1.jpg...
Processing 7/11: R- N- r2-d1.jpg...
Processing 8/11: R- pvc- r3-d1.jpg...
Processing 9/11: R- pe- r1-d1.jpg...
Processing 10/11: R- pe- r3-d1.jpg...
Processing 11/11: R- N- r1-d1.jpg...

Processing complete.
Features saved to: /home/sajad/Sajad_test/Banana/Red/d1/image_features_v3_1.xlsx
Background-removed images saved in: /home/sajad/Sajad_test/Banana/Red/d1/output_bg_removed
Grayscale images (from bg-removed) saved in: /home/sajad/Sajad_test/Banana/Red/d1/output_grayscale
